# Pivot Table

Somewhat like Excel, we can pivot our data using pandas pivot_table functionality. To do so, we will use the `pivot_table()` function.

The `values` parameter is the column being used for aggregation, the `index` parameter is for the index values that creates multiple rows, and the `columns` parameter is for the value on which you want to have multiple columns created.

You can also use the `aggfunc` parameter to pass a function with which to aggregate your pivots.

Let’s look at an example:

In [2]:
import numpy as np
import pandas as pd
names = ['age', 'workclass', 'fnlwgt', 'education', 'educationnum', 'maritalstatus', 'occupation', 'relationship', 'race',
        'sex', 'capitalgain', 'capitalloss', 'hoursperweek', 'nativecountry', 'label']
train_df = pd.read_csv("adult/adult.data", header=None, names=names)

# Pivot the data frame to show by relationship, workclass (rows) and label (columns) the average hours per week.
print(pd.pivot_table(train_df, values='hoursperweek', index=['relationship','workclass'], 
               columns=['label'], aggfunc='mean').round(2))

label                            <=50K   >50K
relationship   workclass                     
Husband        ?                 30.72  37.33
               Federal-gov       42.34  43.05
               Local-gov         41.40  44.56
               Private           42.50  46.18
               Self-emp-inc      48.29  50.49
               Self-emp-not-inc  46.01  48.07
               State-gov         38.67  45.17
               Without-pay       34.25    NaN
Not-in-family  ?                 31.29  39.44
               Federal-gov       40.60  47.54
               Local-gov         40.38  45.01
               Never-worked      35.00    NaN
               Private           40.20  47.03
               Self-emp-inc      49.06  53.58
               Self-emp-not-inc  41.53  45.02
               State-gov         38.87  44.19
Other-relative ?                 29.10  40.00
               Federal-gov       38.40  45.00
               Local-gov         35.92  48.00
               Private           3

Now we have a table of the average hours per week for a given `relationship`, `workclass`, and `label`.

# Cross Tab

`Crosstab` is a nice way to get frequency tables. What you do is pass two columns to the function and you will get the frequency of all the pair-wise combinations of those two variables.

Let’s look at an example using **label** and **relationship** as our columns:

In [3]:
import numpy as np
import pandas as pd
names = ['age', 'workclass', 'fnlwgt', 'education', 'educationnum', 'maritalstatus', 'occupation', 'relationship', 'race',
        'sex', 'capitalgain', 'capitalloss', 'hoursperweek', 'nativecountry', 'label']
train_df = pd.read_csv("adult/adult.data", header=None, names=names)

# Calculate the frequencies between label and relationship
print(pd.crosstab(train_df['label'], train_df.relationship))

relationship  Husband  Not-in-family  Other-relative  Own-child  Unmarried  \
label                                                                        
<=50K            7275           7449             944       5001       3228   
>50K             5918            856              37         67        218   

relationship  Wife  
label               
<=50K          823  
>50K           745  


We now have counts broken down by **relationship** and **label**. The first parameter is for the *rows*, and the second is for the *columns*. We can also normalize the results using the `normalize=True` parameter.

In [4]:
import numpy as np
import pandas as pd
names = ['age', 'workclass', 'fnlwgt', 'education', 'educationnum', 'maritalstatus', 'occupation', 'relationship', 'race',
        'sex', 'capitalgain', 'capitalloss', 'hoursperweek', 'nativecountry', 'label']
train_df = pd.read_csv("adult/adult.data", header=None, names=names)

# Crosstab with normalized outputs
print(pd.crosstab(train_df['label'], train_df.relationship, normalize=True))

relationship   Husband  Not-in-family  Other-relative  Own-child  Unmarried  \
label                                                                         
<=50K         0.223427       0.228771        0.028992   0.153589   0.099137   
>50K          0.181751       0.026289        0.001136   0.002058   0.006695   

relationship      Wife  
label                   
<=50K         0.025276  
>50K          0.022880  


# Reshape
With Pandas, you can use `pivot()` to reshape your data. To illustrate this concept, I will use code from this [post](https://stackoverflow.com/questions/38218063/pivoting-dataframe-with-pandas) to create a dataframe in a long format.

In [6]:
import numpy as np
import pandas as pd

# Create a sample time-based DataFrame similar to tm.makeTimeDataFrame()
dates = pd.date_range("2023-01-01", periods=3)
df_sample = pd.DataFrame(np.random.randn(3, 4), index=dates, columns=list("ABCD"))

def unpivot(frame):
    N, K = frame.shape
    data = {
        'value': frame.values.ravel('F'),
        'variable': np.asarray(frame.columns).repeat(N),
        'date': np.tile(np.asarray(frame.index), K)
    }
    return pd.DataFrame(data, columns=['date', 'variable', 'value'])

df = unpivot(df_sample)
print(df)


         date variable     value
0  2023-01-01        A  0.015784
1  2023-01-02        A  1.605440
2  2023-01-03        A  0.828121
3  2023-01-01        B -0.837930
4  2023-01-02        B -0.901090
5  2023-01-03        B  0.015813
6  2023-01-01        C -0.559858
7  2023-01-02        C -0.209979
8  2023-01-03        C  0.391523
9  2023-01-01        D -0.421125
10 2023-01-02        D -0.029369
11 2023-01-03        D -0.897544


As you can see from the output above, a long format is where you have a variable which acts like a potential column. In this example, **variable** has values A, B, and C. This is a long format. To make it a wide format, we would make columns A, B, and C and remove the variable column.

## Long to Wide Format

In [8]:
import numpy as np
import pandas as pd

# Simulate tm.makeTimeDataFrame() with 3 rows and 4 columns
np.random.seed(0)
dates = pd.date_range("2023-01-01", periods=3)
df_original = pd.DataFrame(np.random.randn(3, 4), index=dates, columns=list("ABCD"))

def unpivot(frame):
    N, K = frame.shape
    data = {
        'value': frame.values.ravel('F'),
        'variable': np.asarray(frame.columns).repeat(N),
        'date': np.tile(np.asarray(frame.index), K)
    }
    return pd.DataFrame(data, columns=['date', 'variable', 'value'])

df = unpivot(df_original)

# Use pivot to reshape back to wide format
df_pivot = df.pivot(index='date', columns='variable', values='value')

print(df_pivot)

variable           A         B         C         D
date                                              
2023-01-01  1.764052  0.400157  0.978738  2.240893
2023-01-02  1.867558 -0.977278  0.950088 -0.151357
2023-01-03 -0.103219  0.410599  0.144044  1.454274


We use the `pivot()` function. The `index` parameter is the column we want for the index of our new dataframe. The `columns` variable is for the column for which we want to use the unique values to create new columns. The `values` parameter is which column we want to use to populate the values of these new columns.

# Wide to Long Format

To convert the format form wide back to long, Pandas provide us with the functionality of unstacking via function `unstack()`. This function (in this situation) will stack our columns back into a single variable.

In [10]:
import numpy as np
import pandas as pd

# Generate sample time-based DataFrame similar to tm.makeTimeDataFrame()
np.random.seed(0)
dates = pd.date_range("2023-01-01", periods=3)
df_original = pd.DataFrame(np.random.randn(3, 4), index=dates, columns=list("ABCD"))

# Create long dataframe
def unpivot(frame):
    N, K = frame.shape
    data = {
        'value': frame.values.ravel('F'),
        'variable': np.asarray(frame.columns).repeat(N),
        'date': np.tile(np.asarray(frame.index), K)
    }
    return pd.DataFrame(data, columns=['date', 'variable', 'value'])

df = unpivot(df_original)

# Convert to wide format
df_pivot = df.pivot(index='date', columns='variable', values='value')

# Convert back to long format using .unstack()
print(df_pivot.unstack())

variable  date      
A         2023-01-01    1.764052
          2023-01-02    1.867558
          2023-01-03   -0.103219
B         2023-01-01    0.400157
          2023-01-02   -0.977278
          2023-01-03    0.410599
C         2023-01-01    0.978738
          2023-01-02    0.950088
          2023-01-03    0.144044
D         2023-01-01    2.240893
          2023-01-02   -0.151357
          2023-01-03    1.454274
dtype: float64
